In [106]:
import os
import pandas as pd

In [ ]:
# 파일명 변경

folder_names = os.listdir('./광주/')

for folder_name in folder_names:
    file_names = os.listdir(f'./광주/{folder_name}')
    for file_name in file_names:
        split_file_name = file_name.replace('.csv','').split('_')
        os.rename(f'./광주/{folder_name}/{file_name}',f'./광주/{folder_name}/{folder_name}_{split_file_name[1]}.csv')

### 데이터프레임 합치기
### 관광지 기본정보

In [ ]:
basic_info = pd.DataFrame()

for folder_name in folder_names:
    df = pd.read_csv(f'./광주/{folder_name}/{folder_name}_관광지 기본정보.csv',encoding='ANSI')
    basic_info = pd.concat([basic_info,df],ignore_index=True)

In [ ]:
basic_info

In [ ]:
basic_info.to_csv('./광주/basic_info.csv',encoding='utf-8',index=False)

### 데이터프레임 합치기

### 내비게이션

In [ ]:
navi_info = pd.DataFrame()

for folder_name in folder_names:
    df = pd.read_csv(f'./광주/{folder_name}/{folder_name}_내비게이션 목적지 검색량 추이.csv',encoding='ANSI',index_col=0)
    df = df.rename(columns={df.columns[0] : folder_name}).transpose()
    df['관광지명'] = folder_name
    navi_info = pd.concat([navi_info,df])

In [ ]:
order = [202401,202302, 202303, 202304,202305, 202306, 202307, 202308,202309, 202310, 202311, 202312, '관광지명']
navi_info = navi_info[order]

In [134]:
order = [202301,202302, 202303, 202304,202305, 202306, 202307, 202308,202309, 202310, 202311, 202312, '관광지명']
navi_info.columns = order

In [136]:
order = ['관광지명',202301,202302, 202303, 202304,202305, 202306, 202307, 202308,202309, 202310, 202311, 202312]
navi_info = navi_info[order]

In [138]:
navi_info.to_csv('./광주/navi_info.csv',encoding='utf-8',index=False)

### 데이터프레임 합치기
### 방문자 거주지별 분포

In [ ]:
order = ['관광지명','광역지자체명', '기초지자체명', '광역지자체별 거주 방문자 비율', '기초지자체별 거주 방문자 비율']

visit_info = pd.DataFrame()

for folder_name in folder_names:
    df = pd.read_csv(f'./광주/{folder_name}/{folder_name}_방문자 거주지별 분포.csv',encoding='ANSI')
    df['관광지명'] = folder_name
    df = df[order]
    
    visit_info = pd.concat([visit_info,df],ignore_index=True)

In [ ]:
visit_info.to_csv('./광주/visit_info.csv',encoding='utf-8',index=False)

### 데이터프레임 합치기
### 성별

In [ ]:
order = ['관광지명','연령대', '남성방문 비율', '여성방문 비율']

gender_info = pd.DataFrame()

for folder_name in folder_names:
    df = pd.read_csv(f'./광주/{folder_name}/{folder_name}_성·연령별 방문자.csv',encoding='ANSI')
    df['관광지명'] = folder_name
    df = df[order]
    gender_info = pd.concat([gender_info,df],ignore_index=True)

In [ ]:
gender_info

,관광지명,연령대,남성방문 비율,여성방문 비율
0,1913송정역시장,70세 이상,4.3,3.0
1,1913송정역시장,60~69세,8.8,5.4
2,1913송정역시장,50~59세,12.5,8.5
3,1913송정역시장,40~49세,9.8,5.9
4,1913송정역시장,30~39세,9.9,7.2
...,...,...,...,...
387,환벽당,40~49세,7.6,7.0
388,환벽당,30~39세,4.2,4.9
389,환벽당,20~29세,4.7,5.2
390,환벽당,10~19세,1.2,1.1


In [ ]:
gender_info.to_csv('./광주/gender_info.csv',encoding='utf-8',index=False)

### 데이터 가져오기
### 위치정보

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

In [ ]:
url = 'https://datalab.visitkorea.or.kr/visualize/getTempleteData.do'

params = {
    "SIDO_CD": 29,
    "SGG_CD": "",
    "BASE_YM1": 2010,
    "BASE_YM2": 2022,
    "CONT_NM": "",
    "KTO_CATE_LCLS_CD": "",
    "KTO_CATE_MCLS_CD": "",
    "firstIndex": float('nan'),
    "pageIndex": 1,
    "recordCountPerPage": 10,
    "orderSelect": "",
    "orderValue": "asc",
    "contentList": "전체",
    "noTheme": "",
    "qid": "LN_05_01_011"
}

response = requests.get(url,params=params)
data = response.json()

In [ ]:
all = []

while True:
    for item in data.get('list', []):
        result = {
            '관광지명': item.get('CONT_NM'),
            '주소': item.get('TAR_ADDR_NM'),
            '대분류': item.get('KTO_CATE_LCLS_NM'),
            '중분류': item.get('KTO_CATE_MCLS_NM'),
            '소분류': item.get('KTO_CATE_SCLS_NM'),
            'longitude': item.get('XCRD'),
            'latitude': item.get('YCRD')
        }
        all.append(result)
        
    params['pageIndex'] += 1
    
    time.sleep(5)
    
    if len(all) == 49:
        break

In [ ]:
df = pd.DataFrame(all)

In [ ]:
df.to_csv('./광주/location_info.csv',encoding='utf-8',index=False)

==================================================================================================

==================================================================================================

==================================================================================================

In [ ]:
import pandas as pd

basic_info = pd.read_csv('./광주/basic_info.csv')
gender_info = pd.read_csv('./광주/gender_info.csv')
location_info = pd.read_csv('./광주/location_info.csv')
navi_info = pd.read_csv('./광주/navi_info.csv')
visit_info = pd.read_csv('./광주/visit_info.csv')

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

stars = []
reviews = []

for i in basic_info['관광지명']:
    url = f'https://www.google.co.kr/maps/search/{i}'
    service = Service(executable_path='./data/chromedriver.exe')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    
    try:
        # 해당 클래스명이 나타날 때까지 최대 10초 대기
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'F7nice'))
        )
        
        star = element.text.split('\n')[0]
        review = element.text.split('\n')[1][1:-1]
        
        stars.append(star)
        reviews.append(review)
    except:
        check = input('일치하는 장소를 클릭 후 y를 눌러주세요(이외 None)')
        if check.upper() == 'Y':
            star = driver.find_element(By.CLASS_NAME, 'F7nice').text.split('\n')[0]
            review = driver.find_element(By.CLASS_NAME, 'F7nice').text.split('\n')[1][1:-1]
            
            stars.append(star)
            reviews.append(review)
        else:
            stars.append(None)
            reviews.append(None)
    
    driver.quit()
    print(f'평점 수 : {len(stars)}','|',f'리뷰 수 : {len(reviews)}','|',f'관광지 명 : {i}','|', f'별점 : {star}','|',f'리뷰 : {review}')
    
    star = None
    review = None

평점 수 : 1 | 리뷰 수 : 1 | 관광지 명 : 1913송정역시장 | 별점 : 3.9 | 리뷰 : 4,489
평점 수 : 2 | 리뷰 수 : 2 | 관광지 명 : 5·18기념공원 | 별점 : 4.4 | 리뷰 : 654
평점 수 : 3 | 리뷰 수 : 3 | 관광지 명 : 5·18민주화운동기록관 | 별점 : 4.1 | 리뷰 : 9
평점 수 : 4 | 리뷰 수 : 4 | 관광지 명 : 고싸움놀이전수회관 | 별점 : 4.1 | 리뷰 : 7
평점 수 : 5 | 리뷰 수 : 5 | 관광지 명 : 광주공원 | 별점 : 4.1 | 리뷰 : 445
평점 수 : 6 | 리뷰 수 : 6 | 관광지 명 : 광주대인시장(대인예술시장) | 별점 : 3.9 | 리뷰 : 1,288
평점 수 : 7 | 리뷰 수 : 7 | 관광지 명 : 광주비엔날레전시관 | 별점 : 4.3 | 리뷰 : 261
평점 수 : 8 | 리뷰 수 : 8 | 관광지 명 : 광주사직공원 | 별점 : 4.3 | 리뷰 : 985
평점 수 : 9 | 리뷰 수 : 9 | 관광지 명 : 광주서강사 | 별점 : None | 리뷰 : None
평점 수 : 10 | 리뷰 수 : 10 | 관광지 명 : 광주송정공원 | 별점 : 4.5 | 리뷰 : 22
평점 수 : 11 | 리뷰 수 : 11 | 관광지 명 : 광주수완호수공원 | 별점 : 4.1 | 리뷰 : 560
평점 수 : 12 | 리뷰 수 : 12 | 관광지 명 : 광주시립미술관 | 별점 : 4.4 | 리뷰 : 1,031
평점 수 : 13 | 리뷰 수 : 13 | 관광지 명 : 광주시민의숲 | 별점 : 3.9 | 리뷰 : 7
평점 수 : 14 | 리뷰 수 : 14 | 관광지 명 : 광주양동시장 | 별점 : 4.1 | 리뷰 : 900
평점 수 : 15 | 리뷰 수 : 15 | 관광지 명 : 광주역사민속박물관 | 별점 : 4.3 | 리뷰 : 407
평점 수 : 16 | 리뷰 수 : 16 | 관광지 명 : 광주예술의거리 | 별점 : 4.1 | 리뷰 : 153
평점 수 : 17 | 

In [ ]:
print(f'총 데이터 수 : {len(basic_info)}')
print('=========================================')
print(f'평점 수 : {len(stars)}')
print(f'리뷰 수 : {len(reviews)}')
print('=========================================')
print(f'총 None값 수 : {stars.count(None)}')
print(f'총 None값 수 : {reviews.count(None)}')

총 데이터 수 : 49
평점 수 : 49
리뷰 수 : 49
총 None값 수 : 2
총 None값 수 : 2


In [ ]:
basic_info['stars'] = stars
basic_info['reviews'] = reviews

In [ ]:
basic_info.to_csv('./광주/basic_info.csv',encoding='utf-8',index=False)

### 리뷰, 체류시간 -> 숫자 값으로 변경

In [ ]:
time = []

for i in basic_info['평균 체류시간']:
    i = i.replace('시간','').replace('분','').strip()
    i = i.split(' ')
    time.append((int(i[0]) * 60) + int(i[1]))
    
basic_info['평균 체류시간'] = time

In [ ]:
reviews_to_int = []

for i in basic_info['reviews']:
    if type(i) == type(None):
        reviews_to_int.append(i)
        continue
    if type(i) in (float,int):
        reviews_to_int.append(i)
    else:
        i = i.replace(',','')
        i = int(i)
        reviews_to_int.append(i)

basic_info['reviews'] = reviews_to_int

In [ ]:
basic_info.to_csv('./광주/basic_info.csv',encoding='utf-8',index=False)

### None값 -> 관광지별 평균

In [ ]:
navi_info.set_index('관광지명',inplace=True)

In [ ]:
navi_info = navi_info.apply(lambda row: row.fillna(int(row.mean())), axis=1)

In [ ]:
navi_info.to_csv('./광주/navi_info.csv',encoding='utf-8',index=False)

### 광역지자체별 거주 방문자 비율

In [ ]:
visit_info.dropna(subset=['광역지자체명'], axis=0,inplace=True)

In [ ]:
names = visit_info['관광지명'].tolist()

local_visit = visit_info[visit_info['광역지자체명'].str.contains('광주')].groupby('관광지명')[['광역지자체별 거주 방문자 비율']].mean()
local_visit = local_visit.reset_index()

local_visit.loc[local_visit['관광지명'].isin(names)]

In [ ]:
basic_info['광역지자체별 거주 방문자 비율'] = local_visit['광역지자체별 거주 방문자 비율']

In [ ]:
basic_info

===============================================================================================================

===============================================================================================================

===============================================================================================================

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

search_number = []
for i in basic_info['관광지명']:
    if not i.startswith('광주'):
        i = '광주 ' + i
    url = f'https://www.google.com/search?q={i}'
    service = Service(executable_path='./data/chromedriver.exe')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    time.sleep(2)

    search_number.append(driver.find_element(By.ID, 'result-stats').text)

basic_info['search_number'] = search_number

In [ ]:
search_number = []
for index,i in enumerate(basic_info['관광지명']):
    if basic_info.iloc[index]['search_number']:
        continue
    if not i.startswith('광주'):
        i = '광주 ' + i
    url = f'https://www.google.com/search?q={i}'
    service = Service(executable_path='./data/chromedriver.exe')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    time.sleep(2)

    # search_number.append(driver.find_element(By.ID, 'result-stats').text)
    basic_info.loc[index,'search_number'] = driver.find_element(By.ID, 'result-stats').text

In [ ]:
import re

pattern = re.compile(r'\([^)]*\)')

basic_info['search_number'] = basic_info['search_number'].apply(lambda x: re.sub(pattern,'',x).strip())

basic_info['search_number'] = basic_info['search_number'].apply(lambda x: int(x.replace('개','').replace(',','')[7:]))

In [ ]:
basic_info.to_csv('./광주/basic_info.csv',encoding='utf-8',index=False)

===============================================================================================================

===============================================================================================================

===============================================================================================================

In [1]:
import pandas as pd

basic_info = pd.read_csv('./광주/basic_info.csv')
gender_info = pd.read_csv('./광주/gender_info.csv')
location_info = pd.read_csv('./광주/location_info.csv')
navi_info = pd.read_csv('./광주/navi_info.csv')
visit_info = pd.read_csv('./광주/visit_info.csv')

In [2]:
totals = []
avgs = []

for i in range(len(navi_info)):
    total = navi_info.iloc[i][1:].sum()
    avg = navi_info.iloc[i][1:].mean()
    
    totals.append(total)
    avgs.append(avg)
navi_info['sum'] = totals
navi_info['avg'] = avgs

In [3]:
df = pd.DataFrame()
df = pd.concat([basic_info,navi_info[['sum','avg']]],axis=1)
df = pd.merge(df,location_info,on='관광지명',how='inner')

In [7]:
import numpy as np
score = []

for i in range(len(df)):
    if pd.isna(df.iloc[i]['stars']):
        star = 0
    else:
        star = df.iloc[i]['stars'] * 1.2
        
    if df.iloc[i]['reviews']:
        review = 0
    else:
        review = np.log1p(max(df['reviews'])) - np.log1p(df.iloc[i]['reviews']) * 1.1
    search = np.log1p(max(df['search_number'])) - np.log1p(df.iloc[i]['search_number'])
    if pd.isna(df.iloc[0]['광역지자체별 거주 방문자 비율']):
        visit = 0
    else:
        visit = np.log1p(df.iloc[i]['광역지자체별 거주 방문자 비율']) * 1.1
    time = np.log1p(df.iloc[i]['평균 체류시간']) * 0.8
    navi = np.log1p(max(df['avg'])) - np.log1p(df.iloc[i]['avg'])
    all = star+review+search+visit+time-navi
    if df.iloc[i]['소분류'] == '산':
        score.append(all*0.9)
    elif df.iloc[i]['소분류'] == '사찰':
        score.append(all*0.9)
    else:
        score.append(all)

df['scores'] = score

In [8]:
first = df.sort_values(by='scores',ascending=False)

In [9]:
first

,관광지명,주소_x,평균 체류시간,stars,reviews,광역지자체별 거주 방문자 비율,search_number,sum,avg,주소_y,대분류,중분류,소분류,longitude,latitude,scores
19,광주호호수생태원,광주광역시 북구,77,4.5,2633.0,54.8,14300,9966.0,830.500000,광주광역시 북구 충효샘길 7(충효동),자연,자연관광지,자연생태관광지,127.001088,35.184080,17.932764
17,광주포충사,광주광역시 남구,113,4.4,382.0,65.1,18800,4949.0,412.416667,광주광역시 남구 포충로 767(원산동),인문(문화/예술/역사),역사관광지,유적지/사적지,126.848746,35.089847,17.330334
46,풍암호수공원,광주광역시 서구,119,4.4,1983.0,66.6,64800,14530.0,1210.833333,광주광역시 서구 풍암동 460,자연,자연관광지,호수,126.870096,35.127732,17.234075
29,빛고을농촌테마공원,광주광역시 남구,92,4.6,157.0,55.8,32900,4491.0,374.250000,광주광역시 남구 포충로 966-15,인문(문화/예술/역사),휴양관광지,테마공원,126.867902,35.084281,16.584203
31,송산유원지,광주광역시 광산구,59,4.0,691.0,59.6,24100,6276.0,523.000000,광주광역시 광산구 송산동 177,인문(문화/예술/역사),휴양관광지,유원지,126.736157,35.158712,16.229983
23,김대중컨벤션센터,광주광역시 서구,85,4.1,2735.0,56.6,595000,96584.0,8048.666667,광주광역시 서구 상무누리로 30 김대중컨벤션센터,인문(문화/예술/역사),문화시설,컨벤션센터,126.840503,35.146762,16.107720
40,전일빌딩245,광주광역시 동구,44,4.7,563.0,74.2,93200,8225.0,685.416667,광주광역시 동구 금남로 245,인문(문화/예술/역사),건축/조형물,유명건물,126.918732,35.148400,15.994767
24,남광주시장,광주광역시 동구,98,3.9,1713.0,63.1,92800,13210.0,1100.833333,광주광역시 동구 제봉로17번길 18-6,쇼핑,쇼핑,상설시장,126.921367,35.139072,15.967402
22,국립아시아문화전당,광주광역시 동구,52,4.6,444.0,69.4,1140000,86901.0,7241.750000,광주광역시 동구 문화전당로 38,인문(문화/예술/역사),문화시설,외국문화원,126.920727,35.147669,15.785362
13,광주양동시장,광주광역시 서구,51,4.1,900.0,74.8,433000,50739.0,4228.250000,광주광역시 서구 천변좌로 238,쇼핑,쇼핑,상설시장,126.902242,35.154288,15.681487


In [11]:
first['관광지명'].tail(25).tolist()

['사직공원전망타워',
 '원효사',
 '원각사(광주)',
 '5·18민주화운동기록관',
 '증심사(광주)',
 '광주사직공원',
 '광주시립미술관',
 '광주역사민속박물관',
 '환벽당',
 '동명동카페거리',
 '광주호',
 '국립광주박물관',
 '충장사',
 '광주수완호수공원',
 '신광사(광주)',
 '5·18기념공원',
 '광주시민의숲',
 '광주송정공원',
 '광주공원',
 '약사암(광주)',
 '푸른길분수공원',
 '오리요리의거리',
 '광주예술의거리',
 '도림사(광주)',
 '광주서강사']

In [166]:
import folium

gwangju_map = folium.Map(location=[35.1599785,126.8513072],zoom_start=11)

In [167]:
for i in range(len(first)):
    if i == 30:
        break
    folium.Marker(
            [first.iloc[i]['latitude'],first.iloc[i]['longitude']],
            popup=folium.Popup(first.iloc[i]['관광지명'],maxWidth=300),
            fill_opacity=0.5,
            icon=folium.Icon(color='blue')            
            ).add_to(gwangju_map)

In [ ]:
gwangju_map